# Data Observation and Data splitting
This notebook includes two parts
* Data observation
* Data splitting

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import re
import os
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Function for summary data

In [18]:
def summary_dataframe(data):
    null_counts = data.isnull().sum()
    summary_df = pd.DataFrame({"column_name":null_counts.index, "null_count":null_counts.values})
    summary_df["null_rate"] = null_counts.values/len(data)
    summary_df["elements"] = [len(set(data[column_name].to_list())) for column_name in data.columns]
    return summary_df

### Setting variable

In [ ]:
DRIVE_MOUNT_PATH = "/content/drive"
PROJECT_PATH = "MyDrive/Fake-JD-Detector"
TARGET_COLUMN = "fraudulent"

### Execut this cell to mount your google drive

In [ ]:
from google.colab import drive
drive.mount(DRIVE_MOUNT_PATH)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Loading Data

In [ ]:
job_postings_file = "job_postings_training_set.csv"
job_postings_df = pd.read_csv(os.path.join(DRIVE_MOUNT_PATH, PROJECT_PATH, job_postings_file))

### Show data top 5 rows

In [ ]:
job_postings_df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


### Summary about null value and distinct elements

In [ ]:
summary_dataframe(job_postings_df)

,column_name,null_count,null_rate,elements
0,job_id,0,0.000000,17828
1,title,0,0.000000,11188
2,location,345,0.019352,3098
3,department,11514,0.645838,1338
4,salary_range,14966,0.839466,875
5,company_profile,3290,0.184541,1708
6,description,1,0.000056,14678
7,requirements,2688,0.150774,11882
8,benefits,7184,0.402962,6132
9,telecommuting,0,0.000000,2


### Show statistic of numerical data

In [ ]:
job_postings_df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
count,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000
mean,8914.500000,0.042966,0.795434,0.491979,0.048351
std,5146.644635,0.202786,0.403395,0.499950,0.214513
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4457.750000,0.000000,1.000000,0.000000,0.000000
50%,8914.500000,0.000000,1.000000,0.000000,0.000000
75%,13371.250000,0.000000,1.000000,1.000000,0.000000
max,17828.000000,1.000000,1.000000,1.000000,1.000000


### Distribution of target column

In [ ]:
job_postings_df[TARGET_COLUMN].value_counts().to_frame()

,fraudulent
0,16966
1,862


## Feature engineering

In [ ]:
# Because of following steps will extend some columns, so copy to new dataframe
job_postings_extend_df = copy.deepcopy(job_postings_df)

In [19]:
# Using 0 length string ("") replace Nan for more easy to detect
job_postings_extend_df.fillna("", inplace=True)

# Add is_null column for all value-loss columns
# If value is "", means loss, set value 1 otherwise 0
null_counts = job_postings_df.isnull().sum()
for column, null_count in null_counts.items():
    if null_count > 0:
        job_postings_extend_df[column+"_is_null"] = job_postings_extend_df[column].apply(lambda x: 0 if len(x) > 0 else 1)

# Spilt location to country, state, city
job_postings_extend_df[["location_country", "location_state", "location_city"]] = job_postings_extend_df["location"].str.split(", ",  2, expand=True)

# Processing salary range to numerical
salary_range_lows, salary_range_ups, salary_range_diff = [], [], []
for salary_range in job_postings_extend_df["salary_range"].tolist():
    
    range_split, range_low, range_up =[], -1, -1
    
    if len(salary_range) > 0:
        range_split = re.sub(r"[^0-9\-]+", "", salary_range).split("-")
    
    if len(range_split) == 2 and len(range_split[0]) > 0 and len(range_split[1]) > 0:
        range_low, range_up = range_split
        
    salary_range_lows.append(int(range_low))
    salary_range_ups.append(int(range_up))
    salary_range_diff.append(int(range_up)-int(range_low))
    
job_postings_extend_df["salary_range_low"] = salary_range_lows
job_postings_extend_df["salary_range_up"] = salary_range_ups
job_postings_extend_df["salary_range_diff"] = salary_range_diff

<ipython-input-19-7c67139ae57b>:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  job_postings_extend_df[["location_country", "location_state", "location_city"]] = job_postings_extend_df["location"].str.split(", ",  2, expand=True)


### Some References Inspire Me

* [Kaggle: Real/Fake Job Posting Prediction](https://www.kaggle.com/datasets/shivamb/real-or-fake-fake-jobposting-prediction)

In [20]:
# Combine string type data to text
job_postings_extend_df["posting_text"] = (
    job_postings_extend_df["title"]+" "+
    job_postings_extend_df["location"]+" "+
    job_postings_extend_df["company_profile"]+" "+
    job_postings_extend_df["description"]+" "+
    job_postings_extend_df["requirements"]+" "+
    job_postings_extend_df["benefits"]
)

# Clean text, only keep a-z, A-Z, 0-9 characters
# 
job_postings_extend_df["posting_text_cleaned"] = job_postings_extend_df["posting_text"].apply(lambda x: re.sub(r"[\s]+", " ", re.sub(r"[^a-zA-Z0-9]+"," ", x)).strip())

In [21]:
#
text_columns = ["company_profile", "description", "requirements", "benefits", "posting_text", "posting_text_cleaned"]
for text_column in text_columns:
    job_postings_extend_df[text_column+"_len"] = job_postings_extend_df[text_column].apply(lambda x: len(x))

In [22]:
summary_dataframe(job_postings_extend_df)

,column_name,null_count,null_rate,elements
0,job_id,0,0.000000,17828
1,title,0,0.000000,11188
2,location,0,0.000000,3098
3,department,0,0.000000,1338
4,salary_range,0,0.000000,875
5,company_profile,0,0.000000,1708
6,description,0,0.000000,14678
7,requirements,0,0.000000,11882
8,benefits,0,0.000000,6132
9,telecommuting,0,0.000000,2


In [23]:
job_postings_extend_df.describe()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent,location_is_null,department_is_null,salary_range_is_null,company_profile_is_null,description_is_null,...,function_is_null,salary_range_low,salary_range_up,salary_range_diff,company_profile_len,description_len,requirements_len,benefits_len,posting_text_len,posting_text_cleaned_len
count,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,...,17828.000000,1.782800e+04,1.782800e+04,1.782800e+04,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000,17828.000000
mean,8914.500000,0.042966,0.795434,0.491979,0.048351,0.019352,0.645838,0.839466,0.184541,0.000056,...,0.360837,8.214686e+04,1.294712e+05,4.732429e+04,621.079594,1218.000224,590.106181,208.925510,2686.940767,2612.368073
std,5146.644635,0.202786,0.403395,0.499950,0.214513,0.137761,0.478272,0.367111,0.387936,0.007489,...,0.480257,7.065713e+06,1.080364e+07,3.750398e+06,567.299039,892.607405,613.566109,337.022023,1464.791575,1423.692278
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-1.000000e+00,-1.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,27.000000,23.000000
25%,4457.750000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,-1.000000e+00,-1.000000e+00,0.000000e+00,139.000000,607.000000,146.000000,0.000000,1612.000000,1570.000000
50%,8914.500000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,-1.000000e+00,-1.000000e+00,0.000000e+00,570.000000,1017.000000,467.000000,45.000000,2540.500000,2473.500000
75%,13371.250000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,-1.000000e+00,-1.000000e+00,0.000000e+00,879.000000,1587.000000,820.000000,294.250000,3490.250000,3404.000000
max,17828.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,8.000000e+08,1.200000e+09,4.000000e+08,6178.000000,14881.000000,10864.000000,4427.000000,14933.000000,14611.000000


## Split Data to Train and Test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    job_postings_extend_df.posting_text_cleaned, 
    job_postings_extend_df.fraudulent, 
    test_size = 0.20, 
    stratify=job_postings_extend_df.fraudulent, 
    random_state = 777
)

train_df = pd.DataFrame({"text": X_train, "fraudulent": y_train})
test_df = pd.DataFrame({"text": X_test, "fraudulent": y_test})

train_df.to_csv("/content/drive/MyDrive/Fake-JD-Detector/splitted_text_data/train.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/Fake-JD-Detector/splitted_text_data/verify.csv", index=False)